In [32]:
import numpy as np
import csv
from sklearn.cross_validation import train_test_split
from scipy.stats.stats import pearsonr
from sklearn.metrics import mean_absolute_error 

In [9]:
import pandas as pd
import graphlab

"""
Read in train and test as Pandas DataFrames
"""
df_profiles = pd.read_csv("profiles.csv")
df_train = pd.read_csv("train.csv")
df_train_with_users = pd.read_csv("train_with_users.csv")

In [13]:
sf_train = graphlab.SFrame(df_train)
# sf_profiles = graphlab.SFrame(df_profiles)
sf_artists = graphlab.SFrame.read_csv("artists.csv")

Finished parsing file /Users/kazuma/Desktop/CS_projects/cs181-s17-kmakihara/181_practicals/cs181-practical3/artists.csv

Parsing completed. Parsed 100 lines in 0.026238 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/kazuma/Desktop/CS_projects/cs181-s17-kmakihara/181_practicals/cs181-practical3/artists.csv

Parsing completed. Parsed 2000 lines in 0.012505 secs.

In [65]:
training_set, validation_set = graphlab.recommender.util.random_split_by_user(sf_train, user_id = 'user', item_id='artist', max_num_users=150000)

In [71]:
popularity_recommender = graphlab.popularity_recommender.create(training_set, user_id = 'user', item_id='artist', target='plays')

Recsys training: model = popularity

Preparing data set.

Data has 3620458 observations with 233286 users and 2000 items.

Data prepared in: 3.8797s

3620458 observations to process; with 2000 unique items.

In [126]:
popularity_recommendations.print_rows(num_rows = 20)

+-------------------------------+-------------------------------+---------------+
|              user             |             artist            |     score     |
+-------------------------------+-------------------------------+---------------+
| eb1c57ddc9e0e2d005169d3a1a... | e69c8ccb-25b1-4c4a-89c6-8f... | 724.307219662 |
| eb1c57ddc9e0e2d005169d3a1a... | b3a70929-32bc-4ace-994f-43... |  714.48951049 |
| eb1c57ddc9e0e2d005169d3a1a... | 5ffb73c0-b739-4231-ba22-f8... | 613.647912886 |
| eb1c57ddc9e0e2d005169d3a1a... | 72d3dc3f-3984-42b9-89de-42... | 612.569491525 |
| eb1c57ddc9e0e2d005169d3a1a... | 92bb085a-2924-4479-b627-18... |  603.94606414 |
| eb1c57ddc9e0e2d005169d3a1a... | 31745282-b1ea-4d62-939f-22... | 603.445549879 |
| eb1c57ddc9e0e2d005169d3a1a... | dcaa4f81-bfb7-44eb-8594-4e... | 593.879752182 |
| eb1c57ddc9e0e2d005169d3a1a... | 99acd557-c4e2-4086-9be9-85... | 574.386084584 |
| eb1c57ddc9e0e2d005169d3a1a... | c0eee88b-47f2-4cd2-ac48-a0... | 568.155752212 |
| eb1c57ddc9e0e2

In [142]:
correct_plays = validation_set.select_column('plays')

In [143]:
validation_set = validation_set.remove_column('plays')

In [153]:
popularity_validation_predictions = popularity_recommender.predict(validation_set)

In [154]:
popularity_validation_predictions

dtype: float
Rows: 534346
[219.97356283046463, 419.60562679833754, 203.4778051787916, 209.2446134347275, 320.59893226559893, 273.9009009009009, 225.25102351152182, 239.02605406065044, 282.29569308722404, 313.53348729792145, 145.92638036809817, 146.53859348198972, 283.25255182573926, 219.97356283046463, 365.84656845131155, 294.83139784946235, 368.14915254237286, 244.22125857200484, 254.96327924850556, 332.1212653778559, 147.40545938748335, 321.3231441048035, 243.75486381322958, 382.3985384708298, 138.45662100456622, 219.97356283046463, 332.6412958902172, 384.8214542836573, 298.67660480681474, 129.88723776223776, 135.9567474048443, 260.0714481248289, 242.37048665620094, 470.9228510411105, 142.49780701754386, 291.66569555717405, 209.8979065098939, 297.41786210087923, 327.13988574776624, 247.6553059643687, 216.9094099178133, 388.0770864567716, 218.5464495461826, 324.599674796748, 198.83506300963677, 285.75530243519245, 303.8185654008439, 215.0746237458194, 172.1479601087942, 220.4474647151

In [158]:
mean_absolute_error(correct_plays, popularity_validation_predictions)

234.23168952860996

In [166]:
similarity_recommender = graphlab.item_similarity_recommender.create(training_set, user_id = 'user', item_id='artist', target='plays', similarity_type='pearson')

Recsys training: model = item_similarity

Preparing data set.

Data has 3620458 observations with 233286 users and 2000 items.

Data prepared in: 3.96647s

Training model from provided data.

...

| 890.282ms                           | 100              | 2000            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.02954s

In [168]:
item_validation_predictions = similarity_recommender.predict(validation_set)

In [169]:
mean_absolute_error(correct_plays, item_validation_predictions)

233.25759375065547

In [186]:
factorization_recommender = graphlab.factorization_recommender.create(training_set, user_id = 'user', item_id='artist', target='plays', num_factors=8, max_iterations=50)

Recsys training: model = factorization_recommender

Preparing data set.

Data has 3620458 observations with 233286 users and 2000 items.

Data prepared in: 3.97792s

Training factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 325927

Final training RMSE: 570.898

In [184]:
factorization_recommender_predictions = factorization_recommender.predict(validation_set)

In [185]:
mean_absolute_error(correct_plays, factorization_recommender_predictions)

265.04924974005689